<a href="https://colab.research.google.com/github/aman06012003/hackathon/blob/main/NER_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

News API

Full function

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time 
from nltk.tokenize import sent_tokenize

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import re

CLEANR = re.compile('<.*?>') 


In [ ]:
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [ ]:
def func(ent,name):
    for x in ent.split(" "):
        if x in name.split(" "):
            return True
    
    return False

In [ ]:
name = input()

Amitesh


In [ ]:
for i in range(df.shape[0]):
  relevant_passages = []

  url = df["link"][i]
  # url = "https://news.google.com/__i/rss/rd/articles/CBMidmh0dHBzOi8vd3d3LmRlY2NhbmhlcmFsZC5jb20vb3Bpbmlvbi9hbmFseXNpbmctYS1tYXN0ZXJzdHJva2UtbW9kaS1nb3Z0LWN1dHMtZm9vZC1hbGxvY2F0aW9uLWZvci10aGUtcG9vci0xMTc3MTQ4Lmh0bWzSAXpodHRwczovL3d3dy5kZWNjYW5oZXJhbGQuY29tL2FtcC9vcGluaW9uL2FuYWx5c2luZy1hLW1hc3RlcnN0cm9rZS1tb2RpLWdvdnQtY3V0cy1mb29kLWFsbG9jYXRpb24tZm9yLXRoZS1wb29yLTExNzcxNDguaHRtbA?oc=5"
  r1 = requests.get(url)


  coverpage = r1.content


  soup1 = BeautifulSoup(coverpage, 'html5lib')


  coverpage_news = soup1.find_all('p')


  coverpage_news = str(coverpage_news)


  coverpage = cleanhtml(coverpage_news)

  text = coverpage
  text = str(text)
  lst = sent_tokenize(text)
  min = []
  flag=0
  for i in lst:
    article = nlp(i)
    if(flag==1):
      for word in article:
        if (word.pos_ == "PRON"):
          min.append(i)
          flag=0
    for ent in article.ents:
      if (ent.label_ == "PERSON" and (ent.text == "Shri Narendra Modi" or func("Shri Narendra Modi",ent.text))):  #and ent.text == "Anthony Albanese"
        min.append(i)
        flag=1
      else:
        flag=0

  min = set(min)
  min = list(min)
  relevant_passages.append(min)



In [ ]:
final_list=[]
for i in relevant_passages:
  for j in i:
    final_list.append(j)

In [ ]:
final_str = ''
for i in final_list:
  final_str+=i

In [ ]:
def summarize(final_str, per=0.6):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary